# Install libraries

In [7]:
!pip install keras_preprocessing

# Import libraries


In [8]:
import ipywidgets as widgets
from IPython.display import display
import pickle
import tensorflow as tf
import re
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

nltk.download("stopwords")
nltk.download("wordnet")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Mount Google Drive


In [9]:
from google.colab import drive
drive.mount("/content/drive")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Load model


In [10]:
lemmatizer = WordNetLemmatizer()
tokenizer = pickle.load(open("/content/drive/MyDrive/FakeNewsDetection/model/ISOT/word2vec/tokenizer", "rb"))
model = tf.keras.models.load_model("/content/drive/MyDrive/FakeNewsDetection/model/ISOT/word2vec/model.h5")

In [11]:
import ipywidgets as widgets
from IPython.display import display, HTML
import matplotlib.pyplot as plt

class UI:
    display(HTML("<style>.red_label { color:red }</style>"))
    display(HTML("<style>.green_label { color:green }</style>"))

    def __init__(self):
        self.news_input = widgets.Text(placeholder='Enter news text')
        self.detect_button = widgets.Button(description='Detect', button_style='success')
        self.output_label = widgets.Label()
        self.output = widgets.Output()

        self.detect_button.on_click(self.detection)

        self.ui = widgets.VBox([
            widgets.Label('Enter news:', style={'font-weight': 'bold'}),
            self.news_input,
            self.detect_button,
            self.output_label,
            self.output,
        ])

    def detection(self, _):
            x_given = self.news_input.value
            x = re.sub("[^a-zA-Z]", " ", x_given)
            x = x.lower()
            x = x.split()
            x = [
                lemmatizer.lemmatize(word)
                for word in x
                if not word in stopwords.words("english")
            ]
            x = tokenizer.texts_to_sequences(x)

            x = list(map(lambda v: v[0] if len(v) > 0 else None, x))
            x = [i for i in x if i is not None]
            for i in range(len(x), 480):
                x = [0] + x

            y_result = model.predict([x])[0][0]
            verdict = "true" if (y_result >= 0.5).astype("int") == 1 else "fake"
            verdict = "Verdict: " + verdict
            text = "The probability of the news to be true is " + str(y_result) + "."
            if (y_result >= 0.5).astype("int") == 1:
                self.output_label.value = verdict
                self.output_label.remove_class("red_label")
                self.output_label.add_class("green_label")
            else:
                self.output_label.value = verdict
                self.output_label.remove_class("green_label")
                self.output_label.add_class("red_label")

ui = UI()
display(ui.ui)


1/1 [==============================] - 0s 91ms/step
